In [2]:
# !pip install --upgrade opencv_jupyter_ui

In [1]:
from myreader import MyReader
import cv2
from opencv_jupyter_ui import cv2_imshow, cv2_destroyAllWindows
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [2]:
reader=MyReader('test')

In [43]:
try:
 reader.seek(700)
 with mp_pose.Pose(static_image_mode=0,
    #                       model_complexity=1, 
    #                       enable_segmentation=True,
                          min_detection_confidence=0.3) as pose:
  while reader.next():
    color=reader.current_color
    depth=reader.current_depth
    depth_color=reader.colorize_current_depth()

    color2=color[500:800,900:1200,:]
    img,res,an=mediapipe_detection(color2,pose)
    cv2_imshow('skeleton',img,'100%')
#     cv2_imshow('color - depth',cv2.hconcat([color,depth_color]),width='100%')
#     cv2_imshow('depth',,width=300)
    print(reader.current_frame,end='\r')
    break
except KeyboardInterupt:pass
    

In [19]:
reader.seek(850)
color=reader.current_color
depth=reader.current_depth
depth_color=reader.colorize_current_depth()
cv2_imshow('color - depth',cv2.hconcat([color,depth_color]),width='100%')
#     cv2_imshow('depth',,width=300)
print(reader.current_frame,end='\r')

In [7]:
cv2.rectangle(color,(980,485),(1000,955),(0,0,255),4)
cv2_imshow('color - depth',cv2.hconcat([color,depth_color]))#,width='100%')

In [8]:
reader.get_distance(980,485,980,955)

1.922576195799822

In [10]:
def mediapipe_detection(image, model):
    result = model.process(image)                 
    mp_drawing.draw_landmarks(
        image,
        result.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
#         landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )
    annotated_image=None
    if result.pose_landmarks:
        annotated_image=image.copy()
        if 0:
            condition = np.stack((result.segmentation_mask,) * 3, axis=-1) > 0.1
            bg_image = np.zeros(image.shape, dtype=np.uint8)
            bg_image[:] = BG_COLOR
            annotated_image = np.where(condition, annotated_image, bg_image)
            annotated_image=trim(annotated_image,condition)
    return image, result,annotated_image


In [20]:
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

with mp_pose.Pose(static_image_mode=0,
#                       model_complexity=1, 
#                       enable_segmentation=True,
                      min_detection_confidence=0.3) as pose:
    img,res,an=mediapipe_detection(color,pose)
    cv2_imshow('skeleton',img,'100%')

[-2.102172613143921, -1.2097043991088867, 3.0]